In [ ]:
# importing all useful functions 
import math
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
import gymnasium as gym
from gymnasium import Env, spaces, register, make
import random
import copy

In [ ]:
class RandomMaze(Env):
    
    
    #----- 1 -----
    #constructor for initialization and some helper functions
    
    
    def __init__(self):
        
        #P is basically State: Action: [ Transition Probability , Next state , Reward , isTerminated?]
        # for actions : 0 -> up 1-> right 2->down 3-> left (clockwise)
        self.P = {
            0: {
                0: [(0.8, 0, -0.04, False),(0.1, 1, -0.04, False),(0.1, 0, -0.04, False)],
                1: [(0.8, 1, -0.04, False),(0.1, 0, -0.04, False),(0.1, 4, -0.04, False)],
                2: [(0.8, 4, -0.04, False),(0.1, 1, -0.04, False),(0.1, 0, -0.04, False)],
                3: [(0.8, 0, -0.04, False),(0.1, 0, -0.04, False),(0.1, 4, -0.04, False)]
            },
            1: {
                0: [(0.8, 1, -0.04, False),(0.1, 2, -0.04, False),(0.1, 0, -0.04, False)],
                1: [(0.8, 2, -0.04, False),(0.1, 1, -0.04, False),(0.1, 6, -0.04, False)],
                2: [(0.8, 1, -0.04, False),(0.1, 2, -0.04, False),(0.1, 0, -0.04, False)],
                3: [(0.8, 0, -0.04, False),(0.1, 1, -0.04, False),(0.1, 1, -0.04, False)]
            },
            2: {
                0: [(0.8, 2, -0.04, False),(0.1, 1, -0.04, False),(0.1, 3,  1.00, True)],
                1: [(0.8, 3,  1.00, True ),(0.1, 2, -0.04, False),(0.1, 6, -0.04, False)],
                2: [(0.8, 6, -0.04, False),(0.1, 1, -0.04, False),(0.1, 3,  1.00, True)],
                3: [(0.8, 1, -0.04, False),(0.1, 2, -0.04, False),(0.1, 6, -0.04, False)]
            },
            3: {
                0: [(1.0, 3, 0.00, True)],         # Goal
                1: [(1.0, 3, 0.00, True)],
                2: [(1.0, 3, 0.00, True)],
                3: [(1.0, 3, 0.00, True)]
            },
            4: {
                0: [(0.8, 0, -0.04, False),(0.1, 4, -0.04, False),(0.1, 4, -0.04, False)],
                1: [(0.8, 4, -0.04, False),(0.1, 0, -0.04, False),(0.1, 8, -0.04, False)],
                2: [(0.8, 8, -0.04, False),(0.1, 4, -0.04, False),(0.1, 4, -0.04, False)],
                3: [(0.8, 4, -0.04, False),(0.1, 0, -0.04, False),(0.1, 8, -0.04, False)]
            },
            5: {
                0: [(1.0, 5, 0.00, True)],
                1: [(1.0, 5, 0.00, True)],         # wall
                2: [(1.0, 5, 0.00, True)],
                3: [(1.0, 5, 0.00, True)]
            },
            6: {
                0: [(0.8, 2, -0.04, False),(0.1, 6, -0.04, False),(0.1, 7, -1.00,  True)],
                1: [(0.8, 7, -1.00, True ),(0.1, 2, -0.04, False),(0.1, 10, -0.04, False)],
                2: [(0.8, 10, -0.04,False),(0.1, 6, -0.04, False),(0.1, 7, -1.00,  True)],
                3: [(0.8, 6, -0.04, False),(0.1, 2, -0.04, False),(0.1, 10, -0.04, False)]
            },
            7: {
                0: [(1.0, 7, 0.00, True)],
                1: [(1.0, 7, 0.00, True)],        # hole
                2: [(1.0, 7, 0.00, True)],
                3: [(1.0, 7, 0.00, True)]
            },
            8: {
                0: [(0.8, 4, -0.04, False),(0.1, 9, -0.04, False),(0.1, 8, -0.04, False)],
                1: [(0.8, 9, -0.04, False),(0.1, 4, -0.04, False),(0.1, 8, -0.04, False)],
                2: [(0.8, 8, -0.04, False),(0.1, 8, -0.04, False),(0.1, 9, -0.04, False)],
                3: [(0.8, 8, -0.04, False),(0.1, 4, -0.04, False),(0.1, 8, -0.04, False)]
            },
            9: {
                0: [(0.8, 9, -0.04, False),(0.1, 10, -0.04, False),(0.1, 8, -0.04, False)],
                1: [(0.8, 10, -0.04, False),(0.1, 9, -0.04, False),(0.1, 9, -0.04, False)],
                2: [(0.8, 9, -0.04, False),(0.1, 8, -0.04, False),(0.1, 10, -0.04, False)],
                3: [(0.8, 8, -0.04, False),(0.1, 9, -0.04, False),(0.1, 9, -0.04, False)]
            },
            10: {
                0: [(0.8, 6, -0.04, False),(0.1, 9, -0.04, False),(0.1, 11, -0.04, False)],
                1: [(0.8, 11, -0.04, False),(0.1, 6, -0.04, False),(0.1, 10, -0.04, False)],
                2: [(0.8, 10, -0.04, False),(0.1, 11, -0.04, False),(0.1, 9, -0.04, False)],
                3: [(0.8, 9, -0.04, False),(0.1, 6, -0.04, False),(0.1, 10, -0.04, False)]
            },
            11: {
                0: [(0.8, 7, -1.00, True ),(0.1, 10, -0.04, False),(0.1, 11, -0.04, False)],
                1: [(0.8, 11, -0.04, False),(0.1, 11, -0.04, False),(0.1, 7, -1.00,  True)],
                2: [(0.8, 11, -0.04, False),(0.1, 10, -0.04, False),(0.1, 11, -0.04, False)],
                3: [(0.8, 10, -0.04,False),(0.1, 11, -0.04, False),(0.1, 7, -1.00,  True)]
            },
        }
        
        self.size = 12 # The size of the grid
        #self.window_size = 512  # The size of the PyGame window
        
        
        #self.agent_location = 8 
        
        # We have 3 observations, corresponding to each position in the 1-D grid
        self.observation_space = spaces.Discrete(self.size)

        # We have 2 actions, corresponding to "left" & "right"
        self.action_space = spaces.Discrete(4)
        self.action_space_size_ = 4 
        
    
    #return the locations of agent and target
    def _get_obs(self):
        return {   
            "agent" : self.agent_location, 
            "target": self._target_location  
        }
    
    #returns the distance between agent and target 
    def _get_info(self):
        return {  
            "distance": abs(self.agent_location - self._target_location)   
        }
    
    
    
    
    #----- 2 ------
    # The reset function to initiate 
    
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.agent_location = 8             #location of agent in the begining
        self._target_location = 3            #location of target  
        self._dead_state = 7                 #dead location
        
        
        observation = self._get_obs()        #getting useful information
        info = self._get_info()

        return observation,info
    
    
    
    
    
    #------- 3 ---------
    # The step function 
    
    def step(self, action):  # takes action as a parameter

        # gets the current location and stores the values from P set 
        prev_location = self.agent_location                                #gets location
        transitions = self.P[prev_location][action]                         #gets the corresponding action tuple
        probabilities, next_states, rewards, terminals = zip(*transitions)  #stores the value for use 
        
        # Randomly select a transition based on the probabilities
        # gives you random state based on your probabilities 
        index = random.choices(range(len(probabilities)), weights=probabilities, k=1)[0]
        # stores the values 
        self.agent_location, reward, terminated = next_states[index], rewards[index], terminals[index]
        
        truncated = False
        observation = self._get_obs()  
        info = self._get_info()

        info["log"] = {"current_state": prev_location, 
                       "action":action,  
                        "next_state": self.agent_location}

        # Return the required 5-tuple
        return observation, reward, terminated, truncated, info


In [ ]:
# Register the custom environment
register(id='RMaze', entry_point=RandomMaze)

optimal_policy = [0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 2] # obtained in previous question 

In [ ]:
def generateTrajectory( Env , policy , maxSteps) :
    observation = environment.reset(seed=0)
    experience = []
    terminated = False
    steps=1
    while not terminated :
        
        
        action = policy[Env.agent_location]
        observation, reward, terminated, truncated, info = environment.step(action)
        
        experience.append( (info['log']['current_state'],info['log']['action'],reward,info['log']['next_state']) )
        
        if terminated or truncated:
            observation = environment.reset(seed=0)
            return experience
            
        if steps > maxSteps :
            observation = environment.reset(seed=0)
            return []
        
        steps+=1
        
environment = make('RMaze')
generateTrajectory(environment , optimal_policy , 10)   

In [ ]:
# decay-type : 0 -> linear decay , 1 -> exponential decay

def decayAlpha(initialValue, finalValue, maxSteps, decayType) :
    ans=[initialValue]
    if decayType == 0 :
        
        steps= abs(finalValue-initialValue)/maxSteps 
        i = 1 
        value = initialValue
        while i <= maxSteps :
            value = value - steps
            ans.append(value)
            i+=1
        return ans
    
    else :
        
        factor =  (finalValue / initialValue) ** (1 / maxSteps) 
        i=1
        
        while i <= maxSteps :
            value = initialValue * (factor ** i)
            ans.append(value)
            i+=1
        
        return ans 
        
y_axis1 = decayAlpha(1000, 10, 10, 1)
y_axis2 = decayAlpha(1000, 10, 10, 0) 

x_axis = [i for i in range(1,12)]


plt.plot(x_axis, y_axis1 , label='exponential')
plt.plot(x_axis, y_axis2 , label='linear'      )

plt.legend()

#plt.savefig("qn22.png", format="png", dpi=300)
plt.show()

In [ ]:
#Monte Carlo First Visit -> firstVisit = True
#Monte Carlo Every Visit -> firstVisit = False 

def MonteCarloPrediction( Env , maxSteps , noEpisodes ,policy ,alph=0.1 , gamma=0.99,   firstVisit=False):
    # initializations
    v = np.zeros(Env.size)
    visited=np.zeros(Env.size)
    v_r = np.zeros((noEpisodes,Env.size))
    G_t = np.zeros(noEpisodes)
    observation = Env.reset(seed=0)
    
    alpha= decayAlpha(alph, 0.01, noEpisodes , 0)
    #algorithm
    for e in range(noEpisodes):
        observation = Env.reset(seed=0)
        value=0
        t = generateTrajectory( Env , policy , maxSteps)
        
        visited[:]=False
        
        for i , (s,action,reward,nex) in enumerate(t):
            if visited[s] and firstVisit :
                continue
            else :
                visited[s]=True
            
            G=0
            j=i
            while j<len(t):
                G += ((gamma**(j-i))*(t[j][2]))
                j+=1
            
            if s==6:
                value=G
            
            v[s]+= alpha[e]*(G-v[s]) 
        G_t[e]=value
        v_r[e]=v
        
    return  v_r


environment = make('RMaze' )
observation = environment.reset(seed=0)
MonteCarloPrediction( environment, 100,5,optimal_policy)

In [ ]:
#Temporal Difference 

def TemporalDifference( Env , noEpisodes ,alph ,policy , gamma=0.99):
    # initializations
    v = np.zeros(Env.size)
    v_r = np.zeros((noEpisodes,Env.size))
    observation = Env.reset(seed=0)
    G_t = np.zeros(noEpisodes)
    alpha= decayAlpha(alph, 0.01, noEpisodes , 1)
    
    
    #algorithm
    for e in range(1,noEpisodes):
        #alpha= decayAlpha(alph, 0.01, e , 1)
        terminated = False 
        value=0
        
        while not terminated :
            action = policy[Env.agent_location]
            observation, reward, terminated, truncated, info = Env.step(action)
            td_target = reward
            if not terminated :
                td_target += gamma * v[info['log']['next_state']]
                
            td_error=td_target-v[info['log']['current_state']] 
            if info['log']['current_state'] == 6 :
                value=td_target
            
            v[info['log']['current_state']]+=(alpha[e]*td_error)
            info['log']['current_state']=info['log']['next_state']
            
        G_t[e]=value
        v_r[e]=v
        observation = Env.reset(seed=0)
    return  v_r


environment = make('RMaze' )
observation = environment.reset(seed=0)
TemporalDifference( environment , 5 , 0.1 , optimal_policy)

In [ ]:
def generateNStepTrajectory( Env , policy , s , n ) :
    observation = environment.reset(seed=0)
    path = []
    terminated = False
    flag = True
    steps=1
    while not terminated :
        
        
        action = policy[s]
        observation, reward, terminated, truncated, info = Env.step(action)
#         if flag:
#             flag=False
#             s_dash=info['log']['next_state']
#             t=terminated
        path.append( reward )
        
        if terminated or truncated or steps > n:
            observation = environment.reset(seed=0)
            return path,info['log']['next_state'],terminated
        
        steps+=1

def calculateReturn(gamma , path , n ):
    sum =0
    c=0
    for i in path:
        if c>=n:
            break
        sum+= ((gamma**c) * i)
        c+=1
        
    return sum    


def n_stepTemporalDifference( Env , policy , n, noEpisodes , gamma=0.99 , alph=0.5 ):
    # initializations
    v = np.zeros(Env.size)
    v_r = np.zeros((noEpisodes,Env.size))
    alpha= decayAlpha(alph, 0.01, noEpisodes , 1)
    G_t = np.zeros(noEpisodes)
    
    #algorithm
    for e in range(noEpisodes):
        
        observation = Env.reset(seed=0)
        s=8                                            #inital pos after reset
        terminated = False 
        path=[]
        
        while not terminated  :
            path, s_dash , terminated = generateNStepTrajectory(Env, policy , s , n)
            target = calculateReturn(gamma , path , n)
            if not terminated :
                target = target + ((gamma**n) * v[s_dash])
                
            ntd_error=target-v[s]
            v[s]=v[s]+(alpha[e] * ntd_error)
            s=s_dash
            if len(path)<=1:
                break
        v_r[e]=v
    return v_r


ev = make('RMaze' )
observation = ev.reset(seed=0)
n_stepTemporalDifference( ev , optimal_policy , 5 , 5 )

In [ ]:
def TDLambdaPrediction(Env , policy , noEpisodes , lamda , gamma=0.99 , alph =1 ):
    v = np.zeros(Env.size)
    v_r = np.zeros((noEpisodes,Env.size))
    E = np.zeros(Env.size)
    y_axis= []
    n=0
    for e in range(noEpisodes):
        alpha = decayAlpha(alph, 0.01, noEpisodes , 1)
        observation = Env.reset(seed=0)
        s=8 
        terminated = False 
        while not terminated:
            action = policy[s]
            observation, reward, terminated, truncated, info = Env.step(action)
            
            td_target = reward 
            if not terminated :
                td_target += (gamma * v[info['log']['next_state']])
            
            td_error = td_target - v[s]
            E[s]+=1
            
            v = v + (alpha[e]*td_error*E)
            E = gamma*lamda*E
            s=info['log']['next_state']
            if e==100 :
                y_axis.append(E)
                n+=1
            
        v_r[e]=v
    return v_r
            
    
ev = make('RMaze' )
observation = ev.reset(seed=0)
TDLambdaPrediction( ev , optimal_policy , 5 , 0.1 )    

In [ ]:
# Normal and Log plot for Monte Carlo FV and EV
#Monte Carlo First Visit -> firstVisit = True
#Monte Carlo Every Visit -> firstVisit = False 
# For getting log plot activate the plt.xscale('log') comment

arr = np.zeros((500,12))
              
for i in range(20): 
    environment = make('RMaze')
    observation = environment.reset(seed=i)
    arr = arr + MonteCarloPrediction( environment, 100,500,optimal_policy,firstVisit = False)
arr = arr/20
y_axis1=arr[:,0]
y_axis2=arr[:,1]
y_axis3=arr[:,2]
y_axis4=arr[:,3]
y_axis5=arr[:,4]
y_axis6=arr[:,5]
y_axis7=arr[:,6]
y_axis8=arr[:,7]
y_axis9=arr[:,8]
y_axis10=arr[:,9]
y_axis11=arr[:,10]
y_axis12=arr[:,11]
y_axis71=y_axis7
x_axis = np.linspace(1,500,500)    

# Plot the data
plt.plot(x_axis, y_axis1, label='state-0')
plt.plot(x_axis, y_axis2, label='state-1')
plt.plot(x_axis, y_axis3, label='state-2')
plt.plot(x_axis, y_axis4, label='state-3')
plt.plot(x_axis, y_axis5, label='state-4')
plt.plot(x_axis, y_axis6, label='state-5')
plt.plot(x_axis, y_axis7, label='state-6')
plt.plot(x_axis, y_axis8, label='state-7')
plt.plot(x_axis, y_axis9, label='state-8')
plt.plot(x_axis, y_axis10, label='state-9')
plt.plot(x_axis, y_axis11, label='state-10')
plt.plot(x_axis, y_axis12, label='state-11')

# Add labels and legend
plt.title('Value vs. Episode')
plt.xlabel('Episode')
plt.ylabel('Value')
plt.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))

plt.axhline(y = -0.223, color = 'blue', linestyle = 'dashed')
plt.axhline(y = 0.107, color = 'orange', linestyle = 'dashed')
plt.axhline(y = 0.90, color = 'green', linestyle = 'dashed')
plt.axhline(y = 0, color = 'red', linestyle = 'dashed')
plt.axhline(y = -0.234, color = 'purple', linestyle = 'dashed')
plt.axhline(y = 0, color = 'brown', linestyle = 'dashed')
plt.axhline(y = 0.318, color = 'pink', linestyle = 'dashed')
plt.axhline(y = 0, color = 'gray', linestyle = 'dashed')
plt.axhline(y = -0.267, color = 'lime', linestyle = 'dashed')
plt.axhline(y = -0.256, color = 'cyan', linestyle = 'dashed')
plt.axhline(y = 0.140, color = 'blue', linestyle = 'dashed')
plt.axhline(y = -0.226, color = 'orange', linestyle = 'dashed')

#plt.xscale('log')

#plt.savefig("qn315.png", format="png", dpi=300)

# Show the plot
plt.show()
    

In [ ]:
# Normal and Log plot for Temporal Difference 
# For getting log plot activate the plt.xscale('log') comment

arr = np.zeros((500,12))
              
for i in range(20): 
    environment = make('RMaze')
    observation = environment.reset(seed=i)
    arr = arr + TemporalDifference( environment , 500 , 0.5 , optimal_policy)
arr = arr/20
y_axis1=arr[:,0]
y_axis2=arr[:,1]
y_axis3=arr[:,2]
y_axis4=arr[:,3]
y_axis5=arr[:,4]
y_axis6=arr[:,5]
y_axis7=arr[:,6]
y_axis8=arr[:,7]
y_axis9=arr[:,8]
y_axis10=arr[:,9]
y_axis11=arr[:,10]
y_axis12=arr[:,11]
y_axis72=y_axis7
x_axis = np.linspace(1,500,500)    

# Plot the data
plt.plot(x_axis, y_axis1, label='state-0')
plt.plot(x_axis, y_axis2, label='state-1')
plt.plot(x_axis, y_axis3, label='state-2')
plt.plot(x_axis, y_axis4, label='state-3')
plt.plot(x_axis, y_axis5, label='state-4')
plt.plot(x_axis, y_axis6, label='state-5')
plt.plot(x_axis, y_axis7, label='state-6')
plt.plot(x_axis, y_axis8, label='state-7')
plt.plot(x_axis, y_axis9, label='state-8')
plt.plot(x_axis, y_axis10, label='state-9')
plt.plot(x_axis, y_axis11, label='state-10')
plt.plot(x_axis, y_axis12, label='state-11')

# Add labels and legend
plt.title('Value vs. Episode')
plt.xlabel('Episode')
plt.ylabel('Value')
plt.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))

plt.axhline(y = -0.223, color = 'blue', linestyle = 'dashed')
plt.axhline(y = 0.107, color = 'orange', linestyle = 'dashed')
plt.axhline(y = 0.90, color = 'green', linestyle = 'dashed')
plt.axhline(y = 0, color = 'red', linestyle = 'dashed')
plt.axhline(y = -0.234, color = 'purple', linestyle = 'dashed')
plt.axhline(y = 0, color = 'brown', linestyle = 'dashed')
plt.axhline(y = 0.318, color = 'pink', linestyle = 'dashed')
plt.axhline(y = 0, color = 'gray', linestyle = 'dashed')
plt.axhline(y = -0.267, color = 'lime', linestyle = 'dashed')
plt.axhline(y = -0.256, color = 'cyan', linestyle = 'dashed')
plt.axhline(y = 0.140, color = 'blue', linestyle = 'dashed')
plt.axhline(y = -0.226, color = 'orange', linestyle = 'dashed')
#plt.xscale('log')

#plt.savefig("qn316.png", format="png", dpi=300)

# Show the plot
plt.show()
    

In [ ]:
# Normal and Log plot for n step Temporal Difference 
# For getting log plot activate the plt.xscale('log') comment

arr = np.zeros((500,12))
              
for i in range(20): 
    environment = make('RMaze')
    observation = environment.reset(seed=i)
    arr = arr + n_stepTemporalDifference( ev , optimal_policy , 12 , 500 )
arr = arr/20
y_axis1=arr[:,0]
y_axis2=arr[:,1]
y_axis3=arr[:,2]
y_axis4=arr[:,3]
y_axis5=arr[:,4]
y_axis6=arr[:,5]
y_axis7=arr[:,6]
y_axis8=arr[:,7]
y_axis9=arr[:,8]
y_axis10=arr[:,9]
y_axis11=arr[:,10]
y_axis12=arr[:,11]
y_axis73=y_axis7
x_axis = np.linspace(1,500,500)    

# Plot the data
plt.plot(x_axis, y_axis1, label='state-0')
plt.plot(x_axis, y_axis2, label='state-1')
plt.plot(x_axis, y_axis3, label='state-2')
plt.plot(x_axis, y_axis4, label='state-3')
plt.plot(x_axis, y_axis5, label='state-4')
plt.plot(x_axis, y_axis6, label='state-5')
plt.plot(x_axis, y_axis7, label='state-6')
plt.plot(x_axis, y_axis8, label='state-7')
plt.plot(x_axis, y_axis9, label='state-8')
plt.plot(x_axis, y_axis10, label='state-9')
plt.plot(x_axis, y_axis11, label='state-10')
plt.plot(x_axis, y_axis12, label='state-11')

# Add labels and legend
plt.title('Value vs. Episode')
plt.xlabel('Episode')
plt.ylabel('Value')
plt.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))

plt.axhline(y = -0.223, color = 'blue', linestyle = 'dashed')
plt.axhline(y = 0.107, color = 'orange', linestyle = 'dashed')
plt.axhline(y = 0.90, color = 'green', linestyle = 'dashed')
plt.axhline(y = 0, color = 'red', linestyle = 'dashed')
plt.axhline(y = -0.234, color = 'purple', linestyle = 'dashed')
plt.axhline(y = 0, color = 'brown', linestyle = 'dashed')
plt.axhline(y = 0.318, color = 'pink', linestyle = 'dashed')
plt.axhline(y = 0, color = 'gray', linestyle = 'dashed')
plt.axhline(y = -0.267, color = 'lime', linestyle = 'dashed')
plt.axhline(y = -0.256, color = 'cyan', linestyle = 'dashed')
plt.axhline(y = 0.140, color = 'blue', linestyle = 'dashed')
plt.axhline(y = -0.226, color = 'orange', linestyle = 'dashed')
#plt.xscale('log')

#plt.savefig("qn311.png", format="png", dpi=300)

# Show the plot
plt.show()
    

In [ ]:
# Normal and Log plot for Temporal Difference lambda prediction
# For getting log plot activate the plt.xscale('log') comment

arr = np.zeros((500,12))
              
for i in range(20): 
    environment = make('RMaze')
    observation = environment.reset(seed=i)
    arr = arr + TDLambdaPrediction( ev , optimal_policy , 500 , 0.1 )    
arr = arr/20
y_axis1=arr[:,0]
y_axis2=arr[:,1]
y_axis3=arr[:,2]
y_axis4=arr[:,3]
y_axis5=arr[:,4]
y_axis6=arr[:,5]
y_axis7=arr[:,6]
y_axis8=arr[:,7]
y_axis9=arr[:,8]
y_axis10=arr[:,9]
y_axis11=arr[:,10]
y_axis12=arr[:,11]
y_axis74=y_axis7
x_axis = np.linspace(1,500,500)    

# Plot the data
plt.plot(x_axis, y_axis1, label='state-0')
plt.plot(x_axis, y_axis2, label='state-1')
plt.plot(x_axis, y_axis3, label='state-2')
plt.plot(x_axis, y_axis4, label='state-3')
plt.plot(x_axis, y_axis5, label='state-4')
plt.plot(x_axis, y_axis6, label='state-5')
plt.plot(x_axis, y_axis7, label='state-6')
plt.plot(x_axis, y_axis8, label='state-7')
plt.plot(x_axis, y_axis9, label='state-8')
plt.plot(x_axis, y_axis10, label='state-9')
plt.plot(x_axis, y_axis11, label='state-10')
plt.plot(x_axis, y_axis12, label='state-11')

# Add labels and legend
plt.title('Value vs. Episode')
plt.xlabel('Episode')
plt.ylabel('Value')
plt.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))

plt.axhline(y = -0.223, color = 'blue', linestyle = 'dashed')
plt.axhline(y = 0.107, color = 'orange', linestyle = 'dashed')
plt.axhline(y = 0.90, color = 'green', linestyle = 'dashed')
plt.axhline(y = 0, color = 'red', linestyle = 'dashed')
plt.axhline(y = -0.234, color = 'purple', linestyle = 'dashed')
plt.axhline(y = 0, color = 'brown', linestyle = 'dashed')
plt.axhline(y = 0.318, color = 'pink', linestyle = 'dashed')
plt.axhline(y = 0, color = 'gray', linestyle = 'dashed')
plt.axhline(y = -0.267, color = 'lime', linestyle = 'dashed')
plt.axhline(y = -0.256, color = 'cyan', linestyle = 'dashed')
plt.axhline(y = 0.140, color = 'blue', linestyle = 'dashed')
plt.axhline(y = -0.226, color = 'orange', linestyle = 'dashed')
#plt.xscale('log')

#plt.savefig("qn318.png", format="png", dpi=300)

# Show the plot
plt.show()
    

In [ ]:
#G_Target MC EV and FV

environment = make('RMaze' )
observation = environment.reset(seed=0)
arr = MonteCarloPrediction( environment,  100,500,optimal_policy ,firstVisit=False)

y_axis=arr


x_axis = np.linspace(1,500,500)    

# Plot the data
plt.scatter(x_axis, y_axis, label='state')


# Add labels and legend
plt.title('MCEV Target vs. Episode')
plt.xlabel('Episode')
plt.ylabel('MCEV')


#plt.xscale('log')

plt.savefig("qn221.png", format="png", dpi=300)

# Show the plot
plt.show()

In [ ]:
environment = make('RMaze' )
observation = environment.reset(seed=0)
arr = TemporalDifference( environment , 500 , 0.5 , optimal_policy)

y_axis=arr


x_axis = np.linspace(1,500,500)    

# Plot the data
plt.scatter(x_axis, y_axis, label='state')

# Add labels and legend
plt.title('TD Target vs. Episode')
plt.xlabel('Episode')
plt.ylabel('TD error')

#plt.xscale('log')

plt.savefig("qn322.png", format="png", dpi=300)

# Show the plot
plt.show()

In [ ]:
ev = make('RMaze' )
observation = ev.reset(seed=0)
n,arr =TDLambdaPrediction( ev , optimal_policy , 500 , 0.1 )   
arr= np.array(arr)

y_axis1=arr[:,0]
y_axis2=arr[:,1]
y_axis3=arr[:,2]
y_axis4=arr[:,3]
y_axis5=arr[:,4]
y_axis6=arr[:,5]
y_axis7=arr[:,6]
y_axis8=arr[:,7]
y_axis9=arr[:,8]
y_axis10=arr[:,9]
y_axis11=arr[:,10]
y_axis12=arr[:,11]



x_axis = np.linspace(1,n,n)    

# Plot the data
plt.plot(x_axis, y_axis1, label='state-0')
plt.plot(x_axis, y_axis2, label='state-1')
plt.plot(x_axis, y_axis3, label='state-2')
plt.plot(x_axis, y_axis4, label='state-3')
plt.plot(x_axis, y_axis5, label='state-4')
plt.plot(x_axis, y_axis6, label='state-5')
plt.plot(x_axis, y_axis7, label='state-6')
plt.plot(x_axis, y_axis8, label='state-7')
plt.plot(x_axis, y_axis9, label='state-8')
plt.plot(x_axis, y_axis10, label='state-9')
plt.plot(x_axis, y_axis11, label='state-10')
plt.plot(x_axis, y_axis12, label='state-11')


# Add labels and legend
plt.title('E vs. Time Step')
plt.xlabel('Time Step')
plt.ylabel('E')
plt.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))
#plt.xscale('log')

plt.savefig("qn313.png", format="png", dpi=300)

# Show the plot
plt.show()

In [ ]:
x_axis = np.linspace(1,500,500)    



plt.plot(x_axis, y_axis71, label='MC')
plt.plot(x_axis, y_axis72, label='TD')
plt.plot(x_axis, y_axis73, label='n-TD')
plt.plot(x_axis, y_axis74, label='TD-l')


plt.xlabel('Episodes')
plt.ylabel('Value')
plt.legend()

#plt.savefig("qn319.png", format="png", dpi=300)

# Show the plot
plt.show()

